**Importi**

In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import math
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Datu ielāde
movies = pd.read_csv("C:\Bakalaurs_praktiskais\Bakalaura-darbs\movies.csv")
ratings = pd.read_csv(r"C:\Bakalaurs_praktiskais\Bakalaura-darbs\ratings.csv")

In [ ]:
#Funkcija, kas noņem gadu no filmu nosaukuma
def extract_year(title):
    try:
        return int(title.strip()[-5:-1])
    except:
        return None

movies['year'] = movies['title'].apply(extract_year)
movies['clean_title'] = movies['title'].apply(lambda x: x.strip()[:-6].strip() if x.strip()[-5:-1].isdigit() else x)

# Ievieto žanrus sarakstā
movies['genres_list'] = movies['genres'].apply(lambda x: x.split('|'))

# Izveido TF-IDF matricu
movies['genres_str'] = movies['genres'].str.replace('|', ' ')

# Inizializē TF-IDF Vectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['genres_str'])

# Aprēķina kosinusa līdzību, izmantojot iebūvēto funkciju cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Izveido maapingu starp filmu indentifikatoriem un nosaukumiem
indices = pd.Series(movies.index, index=movies['title']).drop_duplicates()

In [ ]:
# Uz saturu balstīta metodes funkcija
def get_content_based_recommendations(title, cosine_sim=cosine_sim, movies=movies, indices=indices):

    try:
        idx = indices[title]
    except:
        return pd.DataFrame()
    
    # Iegūst līdzības koeficientu no visām filmām ar padoto filmu
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sakārto filmas balstoties uz līdzības koeficientiem
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Iegūst 10 filmas, kurām ir vislabākais līdzības koeficients
    sim_scores = sim_scores[1:11]
    
    # Iegūst šo filmu indeksus
    movie_indices = [i[0] for i in sim_scores]
    
    # Atgriež top 10 filmas, kuras iegūtas no līdzības matricas
    result = movies.iloc[movie_indices][['movieId', 'title', 'genres']]
    result['similarity_score'] = [i[1] for i in sim_scores]
    return result

# Matricu faktorizācija
# Izveido lietotāju-vienumu matricu
ratings_matrix = ratings.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# # Calculate sparsity
# sparsity = 1.0 - len(ratings) / float(ratings_matrix.shape[0] * ratings_matrix.shape[1])
# print(f"The ratings matrix has {sparsity:.2%} missing values")


# Noramlizē datus, atņemot no datiem datu vidējo vērtību
ratings_mean = ratings_matrix.mean(axis=1)
ratings_demeaned = ratings_matrix.sub(ratings_mean, axis=0)

ratings_demeaned_sparse = ratings_demeaned.to_numpy()

# Latenoto faktoru skaits
num_factors = 50

# Pielieto matricu faktorizāciju, izmantojot SVD
U, sigma, Vt = svds(ratings_demeaned_sparse, k=num_factors)

# Pārvērš "sigma" sigma par diognālu matricu
sigma = np.diag(sigma)

# Prognozē vērtējumus
predicted_ratings = np.dot(np.dot(U, sigma), Vt) + ratings_mean.to_numpy().reshape(-1, 1)
predicted_ratings_df = pd.DataFrame(predicted_ratings, index=ratings_matrix.index, columns=ratings_matrix.columns)

In [7]:
# Uz sadarbību balstīta metodes funkcija
def get_collaborative_recommendations(user_id, predicted_ratings_df=predicted_ratings_df, movies=movies, n=10):

    # Iegūst lietotāja prognozēto vērtējumu
    user_predictions = predicted_ratings_df.loc[user_id]
    
    # Iegūst filmas, kuras lietotājs jau ir vērtējis
    user_rated_movies = ratings[ratings['userId'] == user_id]['movieId'].tolist()
    
    # Izfiltrē filmas, kuras lietotājs jau ir vērtējis
    user_predictions_unrated = user_predictions.drop(user_rated_movies, errors='ignore')
    
    # Sakārto pēc prognozētajiem vērtējumiem
    top_recommendations = user_predictions_unrated.sort_values(ascending=False).head(n)
    
    # Iegūst filmas datus, kuras tiks ieteiktas
    recommended_movies = movies[movies['movieId'].isin(top_recommendations.index)]
    
    # Pievieno prognozēto vērtējumu
    recommended_movies['predicted_rating'] = recommended_movies['movieId'].map(top_recommendations)
    
    # Sakaŗto pēc prognozētājiem vērtējumiem
    recommended_movies = recommended_movies.sort_values('predicted_rating', ascending=False)
    # Atgriež ieteikto filmu sarakstu
    return recommended_movies[['movieId', 'title', 'genres', 'predicted_rating']]

In [10]:
if __name__ == "__main__":
  
    sample_user_id = 1  
    sample_movie = "Toy Story (1995)"
    
    print("\nUz saturu balstīti ieteikumi Recommendations:")
    content_recs = get_content_based_recommendations(sample_movie)
    print(content_recs[['title', 'genres', 'similarity_score']].head())
    
    print("\nUz sadarbību balstīti ieteikumi:")
    collab_recs = get_collaborative_recommendations(sample_user_id)
    print(collab_recs[['title', 'genres', 'predicted_rating']].head())


Uz saturu balstīti ieteikumi Recommendations:
                                               title  \
1706                                     Antz (1998)   
2355                              Toy Story 2 (1999)   
2809  Adventures of Rocky and Bullwinkle, The (2000)   
3000                Emperor's New Groove, The (2000)   
3568                           Monsters, Inc. (2001)   

                                           genres  similarity_score  
1706  Adventure|Animation|Children|Comedy|Fantasy               1.0  
2355  Adventure|Animation|Children|Comedy|Fantasy               1.0  
2809  Adventure|Animation|Children|Comedy|Fantasy               1.0  
3000  Adventure|Animation|Children|Comedy|Fantasy               1.0  
3568  Adventure|Animation|Children|Comedy|Fantasy               1.0  

Uz sadarbību balstīti ieteikumi:
                               title                 genres  predicted_rating
793                  Die Hard (1988)  Action|Crime|Thriller          4.024307
922   